In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=dI-F6foeYFfCqk7cz6fTh3A-M54yktMdXp16QoIkQ8Y&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgsXL53-2zXa-6V1aM_tZP7a-4AEMYsP43KJ4CLIzWGloHIJDbhtfE

Successfully saved authorization token.


In [ ]:
# Mount our Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Tensorflow setup.
!pip install tensorflow==2.4
import tensorflow as tf
print(tf.__version__)
# Folium setup.
import folium
print(folium.__version__)
from os import path as op
import numpy as np
import os

     |████████████████████████████████| 394.7MB 42kB/s 
     |████████████████████████████████| 2.9MB 14.3MB/s 
     |████████████████████████████████| 3.8MB 45.1MB/s 
     |████████████████████████████████| 471kB 43.3MB/s 
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0
2.4.0
0.8.3


In [ ]:
model = tf.keras.models.load_model('./drive/MyDrive/modeloFinalReal.h5')

In [ ]:
# Specify names locations for outputs in Google Drive. 
FOLDER = 'prueba'
ROOT_DIR = './drive/MyDrive/'

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 224
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
BANDS = ['vis-red', 'vis-green', 'vis-blue']

In [ ]:
class Location():
    def __init__(self, longitude, latitude):
        self.longitude = longitude
        self.latitude = latitude

class MyImagen():
    def __init__(self, imagen, region):
        self.imagen = imagen
        self.region = region

In [ ]:
MAX_CLOUD_PROBABILITY = 20

def maskClouds(img):
  clouds = ee.Image(img.get('cloud_mask')).select('probability')
  isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
  return img.updateMask(isNotCloud)

def maskEdges(s2_img):
  return s2_img.updateMask(s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

In [ ]:
name_class = {
    'Barren': 0,
    'Close_Shrubland': 1,
    'Continental_Water': 2,
    'Cropland_seasonal_water': 3,
    'Forest_CDB': 4,
    'Forest_CDN': 5,
    'Forest_CEB': 6,
    'Forest_CEN': 7,
    'Forest_DDB': 8,
    'Forest_DDN': 9,
    'Forest_DEB': 10,
    'Forest_DEN': 11,
    'Forest_ODB': 12,
    'Forest_ODN': 13,
    'Forest_OEB': 14,
    'Forest_OEN': 15,
    'Grassland': 16,
    'Irrigated_Broadleaf_Cropland': 17,
    'Irrigated_Cereal_Cropland': 18,
    'Mangrove': 19,
    'Marine_Water_': 20,
    'Marshland': 21,
    'Moss_and_Lichen': 22,
    'Open_Shrubland': 23,
    'Rainfed_Broadleaf_Cropland': 24,
    'Rainfed_Cereal_Cropland': 25,
    'Snow': 26,
    'Urban': 27
}

class_name = dict((v,k) for k,v in name_class.items())

# Predicciones con getThumbURL

In [ ]:
import urllib.request
from PIL import Image
import cv2
import numpy as np
from googleapiclient import discovery

IMG_HEIGHT = 224
IMG_WIDTH = 224
prueba = False
locations = []

if prueba:
  locations = [Location(-2.42, 36.18), Location(-3.62, 37.19)]
else:
  i = 0
  longitude = -3.62
  latitude = 37.19
  tam = 3
  for i in range(tam):
    for j in range(tam):
        locations.append(Location(longitude + i*0.02, latitude + j*0.02))

images = []
ee_images = []
predicciones = []
probs = []
instances = []
aiplatform = False

for location in locations:
  x=ee.Number(float(location.longitude))
  y=ee.Number(float(location.latitude))
  cor1=(x).subtract(0.0100)
  cor2=(y).subtract(0.0100)
  cor3=(x).add(0.0100)
  cor4=(y).add(0.0100)
  region=ee.Geometry.Rectangle([cor1,cor2,cor3,cor4],None,False);

  s2Sr = ee.ImageCollection('COPERNICUS/S2').filterBounds(region);

  s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(region);

  START_DATE = ee.Date('2015-06-23');
  END_DATE = ee.Date('2020-10-27');
  criteria = ee.Filter.date(START_DATE, END_DATE);

  s2Sr = s2Sr.filter(criteria).map(maskEdges);
  s2Clouds = s2Clouds.filter(criteria);
  s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(**{"primary": s2Sr, "secondary": s2Clouds, "condition": ee.Filter.equals(**{"leftField": 'system:index', "rightField": 'system:index'})});
  s2CloudMaskedPercentile25 =ee.Image(ee.ImageCollection(s2SrWithCloudMask).map(maskClouds).reduce(ee.Reducer.percentile([25]))).unmask(0).clip(region);
  visArgs = {"bands": ["B4_p25", "B3_p25", "B2_p25"], "min": 1, "max": 3558};
  s2CloudMasked_rgb = ee.Image((s2CloudMaskedPercentile25.visualize(**visArgs).copyProperties(s2CloudMaskedPercentile25,s2CloudMaskedPercentile25.propertyNames()))).clip(region)
  image = s2CloudMasked_rgb
  images.append(MyImagen(image, region))
  ee_images.append(image)

  url = image.getThumbURL({'region':region, 'dimensions':224, 'format': 'jpg'})
  print(url)
  encodedURL = urllib.parse.quote(url, safe='://')
  respuesta = urllib.request.urlopen(encodedURL)

  if respuesta:
    generada = Image.open(respuesta)
    img = cv2.cvtColor(np.array(generada), cv2.COLOR_BGR2RGB)

    if aiplatform:
      instances.append(img.tolist())
    else:
      img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH), interpolation = cv2.INTER_AREA)
      img = np.array(img)
      img = img.astype('float32')
      img /= 255 

      x = []
      x.append(img)

      x_array = np.array(x, np.float32)

      pred = model.predict(x_array)

      predicciones.append(np.argmax(pred[0]))
      probs.append(pred[0][np.argmax(pred[0])])

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/d1cace7fe3f00c31fa05e97e90996320-396d983d4046b0b181b0d336aef6f8bf:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/dfa4f26963eee0eb793198da7e24e9a4-d2d911e10047b13174863996895ae97f:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/cbfbc22ebf17f9041e438d1443f96f9b-f9679ed1baad5aec9f0b71f0e5ec06df:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/4858f1f853b9c26242c45413fd622e60-1fead95da92596d0ecc25fe2812ca69e:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/a03e142e2bb8e5d60800624faf23f95b-22bd90139cc3eb533413ead878a1b14e:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/67d8d0ebfb1ed47d706ea538384291e2-bf315c7e14a80d1328315f1debe6b009:getPixels
https://earthengine.googleapis.com/v1alpha/projects/

In [ ]:
imageCol = ee.ImageCollection(ee_images)
# Use folium to visualize the input imagery and the predictions.
mapid = imageCol.getMapId({'bands': ['vis-red', 'vis-green', 'vis-blue'], 'min': 0, 'max': 255})
map = folium.Map(location=[36.18, -2.42], zoom_start=13)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)

map

In [ ]:
if aiplatform:
  project_name = 'tfg-ugr-314011'
  model_name = 'TfgPredictorCustom'
  version_name = 'v0'

  body = {
    'instances': instances
  }

  service = discovery.build('ml', 'v1')
  name = 'projects/{}/models/{}/versions/{}'.format(project_name, model_name, version_name)

  response = service.projects().predict(
      name=name,
      body={'instances': instances}
  ).execute()

  for c in response['predictions']:
    predicciones.append(name_class[c])

  print(predicciones)
else:
  imageCol = ee.ImageCollection(ee_images)
  print(predicciones)
  print(probs)

[27, 27, 27, 27, 27, 24, 25, 25, 25]
[0.9993407, 0.9821698, 0.9223297, 0.99908245, 0.99506986, 0.40050364, 0.4157589, 0.3957318, 0.535721]


In [ ]:
imagesWithPrediction = []
i = 0

for img in images:
  imagesWithPrediction.append(img.imagen.addBands(ee.Image(int(predicciones[i])).clip(img.region)))
  i += 1

In [ ]:
# Use folium to visualize the input imagery and the predictions.
map = folium.Map(location=[locations[0].latitude, locations[0].longitude], zoom_start=5)

landCoverVis = {
    'bands': ['constant'], 
    'min': 0, 
    'max': 27, 
    'palette': ['696969', '006400', '0000ff', '483d8b', 
                'b22222', '008b8b', '9acd32', '00008b', 
                '7f007f', '8fbc8f', 'b03060', 'ff0000',
                'ff8c00', 'ffd700', '7cfc00', '8a2be2',
                '00ffff', '00bfff', '808000', 'da70d6',
                '1e90ff', 'ff00ff', 'ffc0cb', 'add8e6',
                'ff1493', 'ffa07a', 'FFFFFF', 'f0e68c']
}


i = 0
for img in imagesWithPrediction:
  name = class_name[predicciones[i]] + " " + str(probs[i])
  mapid = img.getMapId({'bands': ['vis-red', 'vis-green', 'vis-blue'], 'min': 0, 'max': 255, 'gamma': 1.4})
  folium.TileLayer(
      tiles=mapid['tile_fetcher'].url_format,
      attr='Google Earth Engine',
      overlay=True,
      name=name,
    ).add_to(map)

  prediction = img.getMapId(landCoverVis)
  folium.TileLayer(
      tiles=prediction['tile_fetcher'].url_format,
      attr='Google Earth Engine',
      overlay=True,
      name=name,
    ).add_to(map)

  i += 1

map.add_child(folium.LayerControl())
map

# Predicciones con TIF

In [ ]:
locations = [Location(-2.42, 36.18), Location(-3.62, 37.19)]

images = []
ee_images = []

for location in locations:
  x=ee.Number(float(location.longitude))
  y=ee.Number(float(location.latitude))
  cor1=(x).subtract(0.0100)
  cor2=(y).subtract(0.0100)
  cor3=(x).add(0.0100)
  cor4=(y).add(0.0100)
  region=ee.Geometry.Rectangle([cor1,cor2,cor3,cor4],None,False);

  s2Sr = ee.ImageCollection('COPERNICUS/S2').filterBounds(region);

  s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(region);

  START_DATE = ee.Date('2015-06-23');
  END_DATE = ee.Date('2020-10-27');
  criteria = ee.Filter.date(START_DATE, END_DATE);

  s2Sr = s2Sr.filter(criteria).map(maskEdges);
  s2Clouds = s2Clouds.filter(criteria);
  s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(**{"primary": s2Sr, "secondary": s2Clouds, "condition": ee.Filter.equals(**{"leftField": 'system:index', "rightField": 'system:index'})});
  s2CloudMaskedPercentile25 = ee.Image(ee.ImageCollection(s2SrWithCloudMask).map(maskClouds).reduce(ee.Reducer.percentile([25]))).unmask(0).clip(region);
  visArgs = {"bands": ["B4_p25", "B3_p25", "B2_p25"], "min": 1, "max": 3558};
  s2CloudMasked_rgb = ee.Image((s2CloudMaskedPercentile25.visualize(**visArgs).copyProperties(s2CloudMaskedPercentile25,s2CloudMaskedPercentile25.propertyNames()))).clip(region)
  image = s2CloudMasked_rgb
  images.append(MyImagen(image, region))
  ee_images.append(image)

imageCol = ee.ImageCollection(ee_images)

In [ ]:
# Use folium to visualize the input imagery and the predictions.
mapid = imageCol.getMapId({'bands': ['vis-red', 'vis-green', 'vis-blue'], 'min': 0, 'max': 255})
map = folium.Map(location=[36.18, -2.42], zoom_start=13)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)

map

In [ ]:
original_image_path = "./drive/MyDrive/tif"
!rm -rf original_image_path
!mkdir original_image_path

i = 1
for image in images:
  task = ee.batch.Export.image.toDrive(
    image = image.imagen.select(["vis-red", "vis-green", "vis-blue"]),
    description = ("imagen" + str(i)),
    folder = 'tif',
    region = image.region.getInfo()['coordinates'],
    scale = 10,
    fileFormat = 'GeoTIFF'
  )

  task.start()

  # Block until the task completes.
  print('Running image ' + str(i) + ' export to Google Drive...')
  import time
  while task.active():
      time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
      print('Error with image ' + str(i) + ' export.')
  else:
      print('Image ' + str(i) + ' export completed.')

  i += 1

Running image 1 export to Google Drive...
Image 1 export completed.
Running image 2 export to Google Drive...
Image 2 export completed.


In [ ]:
new_image_path = "./drive/MyDrive/jpg"
#os.rmdir(new_image_path)
#os.mkdir(new_image_path)

In [ ]:
import cv2
from PIL import Image

for r, d, f in os.walk(original_image_path, topdown=False):
    for i in f:
      filename, file_extension = os.path.splitext(i)
      if file_extension == '.tif':
        im = Image.open(original_image_path + '/' + i)
        print("Converting into jpeg the image %s" % i)
        im.thumbnail(im.size)
        i = i.replace('.tif', '.jpg')
        im.save(new_image_path + '/' + i, "JPEG", quality=100)

Converting into jpeg the image imagen1.tif
Converting into jpeg the image imagen2.tif


In [ ]:
import numpy as np

IMG_HEIGHT = 224
IMG_WIDTH = 224

def predictionTif():
  predicciones = []
  probs = []

  for r, d, f in os.walk(new_image_path, topdown=False):
    for i in f:
      filename, file_extension = os.path.splitext(i)
      image_path = (new_image_path + "/" + i)
      img = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
      img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH), interpolation = cv2.INTER_AREA)
      img = np.array(img)
      img = img.astype('float32')
      img /= 255 

      x = []
      x.append(img)

      x_array = np.array(x, np.float32)

      pred = model.predict(x_array)

      predicciones.append(np.argmax(pred[0]))
      probs.append(pred[0][np.argmax(pred[0])])

  return predicciones, probs

[predicciones, probs] = predictionTif()
print(predicciones)
print(probs)

[2, 27]
[0.9976713, 0.9992994]


In [ ]:
imagesWithPrediction = []
i = 0

for img in images:
  imagesWithPrediction.append(img.imagen.addBands(ee.Image(int(predicciones[i])).clip(img.region)))
  i += 1

In [ ]:
# Use folium to visualize the input imagery and the predictions.
map = folium.Map(location=[Latitude, Longitude], zoom_start=5)

landCoverVis = {
    'bands': ['constant'], 
    'min': 0, 
    'max': 27, 
    'palette': ['696969', '006400', '808000', '483d8b', 
                'b22222', '008b8b', '9acd32', '00008b', 
                '7f007f', '8fbc8f', 'b03060', 'ff0000',
                'ff8c00', 'ffd700', '7cfc00', '8a2be2',
                '00ffff', '00bfff', '0000ff', 'da70d6',
                'ff00ff', '1e90ff', 'f0e68c', 'add8e6',
                'ff1493', 'ffa07a', '98fb98', 'ffc0cb']
}


i = 0
for img in imagesWithPrediction:
  mapid = img.getMapId({'bands': ['vis-red', 'vis-green', 'vis-blue'], 'min': 0, 'max': 255, 'gamma': 1.4})
  folium.TileLayer(
      tiles=mapid['tile_fetcher'].url_format,
      attr='Google Earth Engine',
      overlay=True,
      name=class_name[predicciones[i]],
    ).add_to(map)

  prediction = img.getMapId(landCoverVis)
  folium.TileLayer(
      tiles=prediction['tile_fetcher'].url_format,
      attr='Google Earth Engine',
      overlay=True,
      name=class_name[predicciones[i]],
    ).add_to(map)

  i += 1

map.add_child(folium.LayerControl())
map